#### Lock版本的生产者与消费者模式可以正常的运行。但是存在一个不足，在消费者中，总是通过while True死循环并且上锁的方式去判断钱够不够。上锁是一个很耗费CPU资源的行为。因此这种方式不是最好的。
#### 还有一种更好的方式便是使用threading.Condition来实现。threading.Condition可以在没有数据的时候处于阻塞等待状态。一旦有合适的数据了，还可以使用notify相关的函数来通知其他处于等待状态的线程。这样就可以不用做一些无用的上锁和解锁的操作。可以提高程序的性能。
#### 首先对threading.Condition相关的函数做个介绍，threading.Condition类似threading.Lock，可以在修改全局数据的时候进行上锁，也可以在修改完毕后进行解锁。以下将一些常用的函数做个简单的介绍：

#### 1.acquire：上锁。
#### 2.release：解锁。
#### 3.wait：将当前线程处于等待状态，并且会释放锁。可以被其他线程使用notify和notify_all函数唤醒。被唤醒后会继续等待上锁，上锁后继续执行下面的代码。
#### 4.notify：通知某个正在等待的线程，默认是第1个等待的线程。
#### 5.notify_all：通知所有正在等待的线程。notify和notify_all不会释放锁。并且需要在release之前调用。

###### Condition版的生产者与消费者模式代码如下：

In [1]:
import threading
import random
import time

gMoney = 1000
gCondition = threading.Condition()
gTimes = 0
gTotalTimes = 5

class Producer(threading.Thread):
    def run(self):
        global gMoney
        global gCondition
        global gTimes
        while True:
            money = random.randint(100, 1000)
            gCondition.acquire()
            if gTimes >= gTotalTimes:
                gCondition.release()
                print('当前生产者总共生产了%s次'%gTimes)
                break
            gMoney += money
            print('%s当前存入%s元钱，剩余%s元钱' % (threading.current_thread(), money, gMoney))
            gTimes += 1
            time.sleep(0.5)
            gCondition.notify_all()
            gCondition.release()

class Consumer(threading.Thread):
    def run(self):
        global gMoney
        global gCondition
        while True:
            money = random.randint(100, 500)
            gCondition.acquire()
            # 这里要给个while循环判断，因为等轮到这个线程的时候
            # 条件有可能又不满足了
            while gMoney < money:
                if gTimes >= gTotalTimes:
                    gCondition.release()
                    return
                print('%s准备取%s元钱，剩余%s元钱，不足！'%(threading.current_thread(),money,gMoney))
                gCondition.wait()
            gMoney -= money
            print('%s当前取出%s元钱，剩余%s元钱' % (threading.current_thread(), money, gMoney))
            time.sleep(0.5)
            gCondition.release()

def main():
    for x in range(5):
        Consumer(name='消费者线程%d'%x).start()

    for x in range(2):
        Producer(name='生产者线程%d'%x).start()

if __name__ == '__main__':
    main()

<Consumer(消费者线程0, started 5596)>当前取出108元钱，剩余892元钱
<Consumer(消费者线程0, started 5596)>当前取出257元钱，剩余635元钱
<Consumer(消费者线程0, started 5596)>当前取出412元钱，剩余223元钱
<Consumer(消费者线程0, started 5596)>准备取411元钱，剩余223元钱，不足！
<Consumer(消费者线程4, started 6160)>当前取出179元钱，剩余44元钱
<Producer(生产者线程0, started 5992)>当前存入532元钱，剩余576元钱
<Producer(生产者线程0, started 5992)>当前存入416元钱，剩余992元钱
<Producer(生产者线程0, started 5992)>当前存入107元钱，剩余1099元钱
<Producer(生产者线程0, started 5992)>当前存入202元钱，剩余1301元钱
<Producer(生产者线程0, started 5992)>当前存入722元钱，剩余2023元钱
<Consumer(消费者线程4, started 6160)>当前取出283元钱，剩余1740元钱
<Consumer(消费者线程4, started 6160)>当前取出192元钱，剩余1548元钱
<Consumer(消费者线程0, started 5596)>当前取出411元钱，剩余1137元钱
<Consumer(消费者线程1, started 5968)>当前取出260元钱，剩余877元钱
<Consumer(消费者线程1, started 5968)>当前取出403元钱，剩余474元钱
<Consumer(消费者线程1, started 5968)>当前取出362元钱，剩余112元钱
当前生产者总共生产了5次当前生产者总共生产了5次

<Consumer(消费者线程4, started 6160)>当前取出103元钱，剩余9元钱


#### 下同demo5.py文件：

In [2]:
# Condition版的生产者与消费者模式

import threading
import random
import time

gMoney = 1000
gCondition = threading.Condition()
gTotalTimes = 10
gTimes = 0

# 生产者类 (不断生产money)
class Producer(threading.Thread):
    def run(self):
        global gMoney
        global gTimes
        while True:    # 使用while true死循环来不断生产
            money = random.randint(100,1000)
            gCondition.acquire()
            if gTimes >= gTotalTimes:
                gCondition.release()
                break
            gMoney += money
            print("%s生产了%d元钱，剩余%d元钱" %(threading.current_thread(), money, gMoney))
            gTimes += 1
            gCondition.notify_all()
            gCondition.release()
            time.sleep(0.5)

# 消费者类 (断消费money)
class Consumer(threading.Thread):
    def run(self):
        global gMoney
        while True:
            money = random.randint(100,1000)
            gCondition.acquire()
            while gMoney < money:
                if gTimes >= gTotalTimes:
                    gCondition.release()
                    return
                print("%s准备消费%d元钱，剩余%d元钱，余额不足！" %(threading.current_thread(), money, gMoney))
                gCondition.wait()
            gMoney -= money
            print("%s消费了%d元钱，剩余%d元钱" %(threading.current_thread(), money, gMoney))
            gCondition.release()
            time.sleep(0.5)

def main():
    for x in range(5):
        t = Consumer(name="消费者线程%d"%x)   #指定进程的名字
        t.start()

    for x in range(5):
        t = Producer(name="生产者线程%d"%x)   #指定进程的名字
        t.start()

if __name__ == '__main__':
    main()

<Consumer(消费者线程0, started 8032)>消费了471元钱，剩余529元钱
<Consumer(消费者线程1, started 4188)>消费了295元钱，剩余234元钱
<Consumer(消费者线程2, started 6072)>消费了121元钱，剩余113元钱<Consumer(消费者线程3, started 5412)>准备消费781元钱，剩余113元钱，余额不足！
<Consumer(消费者线程4, started 5532)>准备消费481元钱，剩余113元钱，余额不足！<Producer(生产者线程0, started 4136)>生产了633元钱，剩余746元钱<Producer(生产者线程1, started 5108)>生产了312元钱，剩余1058元钱
<Producer(生产者线程2, started 4156)>生产了688元钱，剩余1265元钱


<Producer(生产者线程3, started 7096)>生产了989元钱，剩余2254元钱<Producer(生产者线程4, started 5896)>生产了442元钱，剩余2696元钱<Consumer(消费者线程3, started 5412)>准备消费781元钱，剩余577元钱，余额不足！
<Consumer(消费者线程4, started 5532)>消费了481元钱，剩余577元钱



<Consumer(消费者线程3, started 5412)>消费了781元钱，剩余1915元钱
<Consumer(消费者线程0, started 8032)>消费了461元钱，剩余1454元钱
<Consumer(消费者线程1, started 4188)>消费了698元钱，剩余756元钱
<Consumer(消费者线程2, started 6072)>消费了355元钱，剩余401元钱
<Producer(生产者线程0, started 4136)>生产了584元钱，剩余985元钱
<Producer(生产者线程1, started 5108)>生产了468元钱，剩余1453元钱
<Consumer(消费者线程4, started 5532)>消费了284元钱，剩余1169元钱
<Producer(生产者线程2, started 4156)>生产了614元钱